In [13]:
import pandas as pd
# from pyspark.sql.utils import AnalysisException
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import sys
# sys.path.append(r'/mnt/proj/adas-daas/common_libraries/adas_mtd/')
# from src.data.utils import db_helper as dbh

In [14]:
config_path = '../config.yaml'
from scripts.dataloader import OnlineSpark
session = OnlineSpark(config_path,size='large')
session.path_dict

{'sut': {'root': 's3a://vasp-got-da/data/spark/rwup/sut/',
  'flc_obj': 's3a://vasp-got-da/data/spark/rwup/sut/record_id*flc*R5_object*',
  'vehicle': 's3a://vasp-got-da/data/spark/rwup/sut/record_id*vehicle*'},
 'avl_reference': {'root': 's3a://vasp-got-da/data/spark/rwup/avl_reference/',
  'ego': 's3a://vasp-got-da/data/spark/rwup/avl_reference/record_id*_ego_df*',
  'line': 's3a://vasp-got-da/data/spark/rwup/avl_reference/record_id*_line_df*',
  'object': 's3a://vasp-got-da/data/spark/rwup/avl_reference/record_id*_object_df*'}}

Scenario (indicator):                                            \
Emergency brake (abs(acceleration)>threshold)                    \
Low speed when traffic congestion (0<=velocity<threshold)        \
Lane changing (Lane changing indicators on)                      \
Turning (yaw rate>threshold)                                     \
~~On/off highway (timestamp)~~ 

In [15]:
# Filter-highway


ego = session.loadParquet('avl_reference','ego')

spark = session.spark

csv_file_path = 'data/highway.csv'
df = pd.read_csv(csv_file_path)
csv_df = spark.createDataFrame(df)

csv_df = csv_df.withColumn("start_time", F.col("start_time").cast("double"))
csv_df = csv_df.withColumn("end_time", F.col("end_time").cast("double"))

filtered_df = ego.join(
    csv_df,
    (ego.recording_id == csv_df.recording_id) &
    (F.expr(f"ptp_time >= start_time") & F.expr(f"ptp_time <= end_time")),
    "inner"
)

filtered_df = filtered_df.drop(csv_df.recording_id,csv_df.start_time,csv_df.end_time)
filtered_df = filtered_df.withColumn("Velocity", F.sqrt(F.pow(filtered_df.VelocityX, 2) + F.pow(filtered_df.VelocityY, 2)))
filtered_df = filtered_df.withColumn("ptp_time", F.round(filtered_df["ptp_time"], 1))

In [11]:
obj = session.loadParquet('avl_reference','object')

filtered_obj = obj.join(
    csv_df,
    (obj.recording_id == csv_df.recording_id) &
    (F.expr(f"ptp_time >= start_time") & F.expr(f"ptp_time <= end_time")),
    "inner"
)

filtered_obj = filtered_obj.drop(csv_df.recording_id,csv_df.start_time,csv_df.end_time)

In [12]:
# print(
#     "Obj Vx: {}".format(filtered_obj.approxQuantile("VelocityX", [0.25, 0.75], 0.01)),
#     "Obj Vy: {}".format(filtered_obj.approxQuantile("VelocityY", [0.25, 0.75], 0.01)),
#     "Ego Vx: {}".format(filtered_df.approxQuantile("VelocityX", [0.25, 0.75], 0.01)),
#     "Ego Vy: {}".format(filtered_df.approxQuantile("VelocityY", [0.25, 0.75], 0.01))
# )

In [13]:
obj_columns = ["ptp_time","MessageNumber","ReferenceType","TrackingID","ExistConf","LifeTime","MeasurementState","Length","Width","Height","PositionX","PositionY","PositionZ","Roll","Pitch","Yaw","VelocityX","VelocityY","VelocityZ","AccelerationX","AccelerationY","AccelerationZ","ReferencePoint","MovementState","ClassificationProbability","ObjType","ObjVehicleClassification","recording_id","vehicle"]
filtered_obj = filtered_obj.filter((F.abs(filtered_obj.VelocityX) < 200) & (F.abs(filtered_obj.VelocityY) <200) & (F.abs(filtered_obj.VelocityZ) <200) & (filtered_obj.VelocityX > -1)&(filtered_obj.PositionY < 5)&(filtered_obj.PositionY > -5)&(filtered_obj.PositionX < 40)).dropDuplicates(obj_columns)
filtered_obj = filtered_obj.withColumn("ptp_time", F.round(filtered_obj["ptp_time"], 1))

In [14]:
filtered_obj.show(20)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [15]:
ego_obj = filtered_obj.join(
    filtered_df,
    (filtered_obj.ptp_time == filtered_df.ptp_time)&(filtered_obj.recording_id == filtered_df.recording_id),
    "inner"
)

ego_obj = ego_obj.drop(filtered_df.ptp_time, filtered_df.VelocityX, filtered_df.recording_id)

In [ ]:
ego_obj.show()

+--------------+-------------+-------------+----------+------------------+-------------------+----------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------+-------------+-------------------------+-------+------------------------+--------------------+-------+----------+-----------------+-------+--------------+-------------+-------------+-------------------+-------------------+-------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+----------+-----------------+------------------+
|   

In [ ]:
# sample_plot = filtered_obj.sample(fraction=0.001).toPandas()

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # filtered_data = sample_plot[(sample_plot['PositionX'] >= 0)&(sample_plot['PositionX'] <= 25)]
# # sns.histplot(filtered_data['PositionX'], kde=True)
# sns.histplot(sample_plot['PositionZ'], kde=True)
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.show()

In [16]:
status = session.loadParquet('sut','vehicle')

filtered_status = status.join(
    csv_df,
    (status.recording_id == csv_df.recording_id) &
    (F.expr(f"ptp_time >= start_time") & F.expr(f"ptp_time <= end_time")),
    "inner"
)

filtered_status = filtered_status.drop(csv_df.recording_id,csv_df.start_time,csv_df.end_time)

BrakePedalPosition BrakeSwitch BrakeLight_rqst LateralAcceleration LongitudinalAcceleration VehicleDirSAE_VehSpeed WheelBasedVehicleSpeed SpeedAdaptEnableRqst_SlowTraffic LaneChangeStatus TurnSignalStatus YawRate SteeringWheelAngle

In [18]:
useful_columns = [
    "ptp_time_rounded","recording_id","vehicle","BrakePedalPosition", "BrakeSwitch", 
    "BrakeLight_rqst", "LateralAcceleration", "LongitudinalAcceleration", "VehicleDirSAE_VehSpeed",
    "WheelBasedVehicleSpeed", "LaneChangeStatus", "YawRate", "SteeringWheelAngle"
]

useful_status = filtered_status.select(*useful_columns)
useful_status = useful_status.withColumn("ptp_time_rounded", F.round(useful_status["ptp_time_rounded"], 1))

In [19]:
useful_status.count()

7878349

In [20]:
lane_change = useful_status.filter((useful_status['LaneChangeStatus']!=0)&(useful_status['VehicleDirSAE_VehSpeed']>30)).select("ptp_time_rounded","recording_id").withColumnRenamed("ptp_time_rounded", "ptp_time").withColumn("status",F.lit(1)).dropDuplicates(["ptp_time", "recording_id"])

In [21]:
lane_change.count()
# 254min

63001

In [19]:
emergency_brake = useful_status.filter((useful_status['LaneChangeStatus']==0)&(useful_status['BrakeSwitch']!=0)&(useful_status['BrakePedalPosition']>15)&(useful_status['VehicleDirSAE_VehSpeed']>30)&(useful_status['BrakeLight_rqst']!=0)).select("ptp_time_rounded","recording_id").withColumnRenamed("ptp_time_rounded", "ptp_time").withColumn("status",F.lit(2)).dropDuplicates(["ptp_time", "recording_id"])

In [ ]:
emergency_brake.count()

119515

In [20]:
turning = useful_status.filter((useful_status['LaneChangeStatus']==0)&(F.abs(useful_status['SteeringWheelAngle'])>0.4)&(useful_status['VehicleDirSAE_VehSpeed']>60)).select("ptp_time_rounded","recording_id").withColumnRenamed("ptp_time_rounded", "ptp_time").withColumn("status",F.lit(3)).dropDuplicates(["ptp_time", "recording_id"])

In [ ]:
turning.count()

121679

In [21]:
def dtp(datetime): #datetime to ptp time
    return pd.to_datetime(datetime,unit='ns').timestamp()

def ptd(ptp_time): #ptptime to datetime
    return pd.to_datetime(ptp_time,unit='s')

In [22]:
num_obj1 = ego_obj.groupBy("recording_id", "ptp_time").agg(
    F.count("*").alias("count"),
    F.avg("VelocityX").alias("avg_v"),
    F.avg("Velocity").alias("ego_v"),
)

In [ ]:
num_obj1.filter((num_obj1['count']>4)&(num_obj1['avg_v']<15)&(num_obj1['ego_v']<15)).count()

65034

In [23]:
congestion = num_obj1.filter((num_obj1['count']>4)&(num_obj1['avg_v']<15)&(num_obj1['ego_v']<15)).select("ptp_time","recording_id").withColumn("status",F.lit(4)).dropDuplicates(["ptp_time", "recording_id"])

lane_change \
emergency_brake \
turning \
congestion

In [24]:
new_df = filtered_obj.withColumn("status", F.lit(0))

new_df = new_df.join(lane_change.withColumnRenamed("status", "lane_change_status"),
                     on=["ptp_time", "recording_id"], how="left") \
               .withColumn("status", F.coalesce(F.col("lane_change_status"), F.col("status"))) \
               .drop("lane_change_status")

new_df = new_df.join(emergency_brake.withColumnRenamed("status", "emergency_brake_status"),
                     on=["ptp_time", "recording_id"], how="left") \
               .withColumn("status", F.coalesce(F.col("emergency_brake_status"), F.col("status"))) \
               .drop("emergency_brake_status")

new_df = new_df.join(turning.withColumnRenamed("status", "turning_status"),
                     on=["ptp_time", "recording_id"], how="left") \
               .withColumn("status", F.coalesce(F.col("turning_status"), F.col("status"))) \
               .drop("turning_status")

new_df = new_df.join(congestion.withColumnRenamed("status", "congestion_status"),
                     on=["ptp_time", "recording_id"], how="left") \
               .withColumn("status", F.coalesce(F.col("congestion_status"), F.col("status"))) \
               .drop("congestion_status")


In [ ]:
new_df.show()

+--------------+--------------------+-------------+-------------+----------+------------------+-------------------+----------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------+-------------+-------------------------+-------+------------------------+-------+----------+-----------------+------+
|      ptp_time|        recording_id|MessageNumber|ReferenceType|TrackingID|         ExistConf|           LifeTime|MeasurementState|            Length|             Width|            Height|         PositionX|          PositionY|          PositionZ|                Roll|               Pitch|                 Yaw|           VelocityX|           VelocityY|           VelocityZ|      AccelerationX|       AccelerationY|      Acceleratio

In [ ]:
filtered_obj.count()

3859759

In [ ]:
new_df.count()

3859759

In [ ]:
new_df.filter(new_df['status']!=0).count()

523629

In [ ]:
num_veh = filtered_df.groupBy("vehicle").count()

In [ ]:
num_veh.show()

+-------+-------+
|vehicle|  count|
+-------+-------+
| FH2467|2087954|
| FH2369|1577180|
| FM1073|2784313|
| UDM523| 583110|
+-------+-------+



In [29]:
new_df = new_df.join(
    useful_status.withColumnRenamed("ptp_time_rounded", "ptp_time").select("ptp_time", "recording_id", "VehicleDirSAE_VehSpeed"),
    on=["ptp_time", "recording_id"],
    how="left"
)
new_df.drop(useful_status["ptp_time_rounded"],useful_status["recording_id"])

DataFrame[ptp_time: double, recording_id: string, MessageNumber: bigint, ReferenceType: string, TrackingID: double, ExistConf: double, LifeTime: double, MeasurementState: double, Length: double, Width: double, Height: double, PositionX: double, PositionY: double, PositionZ: double, Roll: double, Pitch: double, Yaw: double, VelocityX: double, VelocityY: double, VelocityZ: double, AccelerationX: double, AccelerationY: double, AccelerationZ: double, ReferencePoint: double, MovementState: double, ClassificationProbability: double, ObjType: double, ObjVehicleClassification: double, vehicle: string, sw_version: string, __index_level_0__: bigint, status: int, VehicleDirSAE_VehSpeed: double, VehicleDirSAE_VehSpeed: double]

In [32]:
new_df = new_df.drop(useful_status["VehicleDirSAE_VehSpeed"])

In [33]:
new_df.show()

+--------------+--------------------+-------------+-------------+----------+------------------+-------------------+----------------+------------------+------------------+------------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+-------------+-------------------------+-------+------------------------+-------+----------+-----------------+------+----------------------+----------------------+
|      ptp_time|        recording_id|MessageNumber|ReferenceType|TrackingID|         ExistConf|           LifeTime|MeasurementState|            Length|             Width|            Height|         PositionX|          PositionY|          PositionZ|                Roll|               Pitch|                 Yaw|        VelocityX|          VelocityY|          VelocityZ|      AccelerationX|   

In [ ]:
new_df.select()

In [11]:
from src.scenario import get_scenarios as gs

In [12]:
finaldf = gs.get_scenarios()

NameError: name 'pd' is not defined